In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [24]:
import random
import time

# Function to generate the initial Tenner grid
def generate_tenner_grid():
    rows, cols = 3, 10
    grid = [[None for _ in range(cols)] for _ in range(rows)]  # Initialize with None (no negative values)

    # Function to check if placement is valid
    def valid_placement(grid, r, c, num):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, -1), (-1, 1), (1, 1)]
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if grid[nr][nc] == num:
                    return False
        return True

    # Place numbers 0-9 in each row satisfying all conditions
    for r in range(rows):
        available_numbers = list(range(cols))
        for c in range(cols):
            valid_numbers = [num for num in available_numbers if valid_placement(grid, r, c, num)]
            if not valid_numbers:
                return generate_tenner_grid()  # Restart grid generation if stuck
            choice = random.choice(valid_numbers)
            grid[r][c] = choice
            available_numbers.remove(choice)

    print("Full grid before deleting cells:")
    print_grid(grid)

    # Calculate column sums
    column_sums = [sum(grid[r][c] for r in range(rows) if grid[r][c] is not None) for c in range(cols)]

    # Randomly empty some cells
    num_empty_cells = random.randint(5, 15)  # Randomly empty between 5 and 15 cells
    empty_positions = set()
    while len(empty_positions) < num_empty_cells:
        empty_r = random.randint(0, rows - 1)
        empty_c = random.randint(0, cols - 1)
        empty_positions.add((empty_r, empty_c))
    for r, c in empty_positions:
        grid[r][c] = None  # Use None to represent empty cells

    print("Grid after randomly deleting cells:")
    print_grid(grid)
    print('-----------------------------------------------')

    return grid, column_sums

# Function to print the grid
def print_grid(grid):
    for row in grid:
        print(' '.join(str(cell) if cell is not None else '.' for cell in row))

# Class to represent a Cell
class Cell:
    def __init__(self, initial_value):
        self.value = initial_value
        if initial_value is None:
            self.domain = list(range(10))  # Values from 0 to 9
        else:
            self.domain = [initial_value]  # The domain is the initial value if it's not None

# Simple Backtracking Solver
class TennerGridB:
    def __init__(self, initial_grid, column_sums):
        self.rows = len(initial_grid)
        self.cols = len(initial_grid[0])
        self.grid = [[Cell(initial_grid[r][c]) for c in range(self.cols)] for r in range(self.rows)]
        self.column_sums = column_sums
        self.consistency_checks = 0
        self.assignment_count = 0

    def is_safe(self, row, col, num):
        # Check row uniqueness
        for i in range(self.cols):
            if i != col and self.grid[row][i].value == num:
                self.consistency_checks += 1
                return False

        # Check column sum
        current_sum = sum(self.grid[r][col].value for r in range(self.rows) if self.grid[r][col].value is not None)
        if current_sum + num > self.column_sums[col]:
            self.consistency_checks += 1
            return False

        # Check adjacent cells
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, 1), (-1, -1), (1, 1), (1, -1)]
        for dr, dc in directions:
            nr, nc = row + dr, col + dc
            if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr][nc].value == num:
                self.consistency_checks += 1
                return False

        return True

    def solve(self, row=0, col=0):
        if col == self.cols:
            col = 0
            row += 1
        if row == self.rows:
            return True

        cell = self.grid[row][col]
        if cell.value is not None:
            return self.solve(row, col + 1)

        for value in cell.domain[:]:  # Work on a copy to avoid modifying the original while iterating
            if self.is_safe(row, col, value):
                cell.value = value
                self.assignment_count += 1
                if self.solve(row, col + 1):
                    return True
                cell.value = None  # Backtrack

        return False

    def print_state(self):
        print('Grid:')
        for row in self.grid:
            print(' '.join(str(cell.value) if cell.value is not None else '.' for cell in row))
        print("Column sums:", self.column_sums)
        print('Assignments:', self.assignment_count)
        print('Consistency checks:', self.consistency_checks)

# Forward Checking Solver
class TennerGridFC(TennerGridB):
    def forward_check(self, row, col, value):
        removals = []
        # Remove the assigned value from the domains of all unassigned cells in the row and column
        for r in range(self.rows):
            if r != row and value in self.grid[r][col].domain:
                self.grid[r][col].domain.remove(value)
                removals.append((r, col, value))
        for c in range(self.cols):
            if c != col and value in self.grid[row][c].domain:
                self.grid[row][c].domain.remove(value)
                removals.append((row, c, value))
        return removals

    def restore_domains(self, removals):
        for r, c, value in removals:
            if value not in self.grid[r][c].domain:
                self.grid[r][c].domain.append(value)

    def solve(self, row=0, col=0):
        if col == self.cols:
            col = 0
            row += 1
        if row == self.rows:
            return True

        cell = self.grid[row][col]
        if cell.value is not None:
            return self.solve(row, col + 1)

        original_domain = cell.domain[:]
        for value in original_domain:
            if self.is_safe(row, col, value):
                cell.value = value
                self.assignment_count += 1
                removals = self.forward_check(row, col, value)
                if self.solve(row, col + 1):
                    return True
                # Backtrack
                cell.value = None
                self.restore_domains(removals)

        return False

# Forward Checking with MRV Solver
class TennerGridMRV(TennerGridFC):
    def find_mrv_cell(self):
        min_domain_size = float('inf')
        min_cell = None
        for r in range(self.rows):
            for c in range(self.cols):
                if self.grid[r][c].value is None and len(self.grid[r][c].domain) < min_domain_size:
                    min_domain_size = len(self.grid[r][c].domain)
                    min_cell = (r, c)
        return min_cell

    def solve(self):
        cell = self.find_mrv_cell()
        if cell is None:  # Check if no unassigned cells are left
            return True

        row, col = cell
        for num in self.grid[row][col].domain[:]:
            if self.is_safe(row, col, num):
                self.grid[row][col].value = num
                self.assignment_count += 1
                removals = self.forward_check(row, col, num)

                if self.solve():
                    return True

                # Backtrack and restore domains
                self.grid[row][col].value = None
                self.restore_domains(removals)

        return False

def main():
    # Generate initial grid and column sums
    initial_grid, column_sums = generate_tenner_grid()

    # Simple Backtracking Solver
    print('Initial State:')
    grid = TennerGridB(initial_grid, column_sums)
    grid.print_state()
    print('-----------------------------------------------')
    print('Simple Backtracking')

    start_time = time.perf_counter()
    sol = grid.solve()
    finish_time = time.perf_counter()

    if sol:
        print('Solved State:')
        grid.print_state()
    else:
        print('No solution found.')

    elapsed_time = finish_time - start_time
    print(f'Elapsed time: {elapsed_time:.6f} seconds')
    print('-----------------------------------------------')

    # Forward Checking Solver
    print('Forward Checking')
    grid = TennerGridFC(initial_grid, column_sums)
    start_time = time.perf_counter()
    sol = grid.solve()
    finish_time = time.perf_counter()

    if sol:
        print('Solved State:')
        grid.print_state()
    else:
        print('No solution found.')

    elapsed_time = finish_time - start_time
    print(f'Elapsed time: {elapsed_time:.6f} seconds')
    print('-----------------------------------------------')

    # Forward Checking with MRV Solver
    print('Forward Checking with MRV')
    grid = TennerGridMRV(initial_grid, column_sums)
    start_time = time.perf_counter()
    sol = grid.solve()
    finish_time = time.perf_counter()

    if sol:
        print('Solved State:')
        grid.print_state()
    else:
        print('No solution found.')

    elapsed_time = finish_time - start_time
    print(f'Elapsed time: {elapsed_time:.6f} seconds')
    print('-----------------------------------------------')

if __name__ == "__main__":
    main()


Full grid before deleting cells:
1 7 6 3 0 5 9 2 8 4
6 0 9 8 1 3 4 7 5 2
1 3 5 0 4 7 2 9 6 8
Grid after randomly deleting cells:
. 7 . 3 0 . 9 2 . .
6 . 9 . . . 4 7 5 2
1 3 5 0 4 . 2 . 6 8
-----------------------------------------------
Initial State:
Grid:
. 7 . 3 0 . 9 2 . .
6 . 9 . . . 4 7 5 2
1 3 5 0 4 . 2 . 6 8
Column sums: [8, 10, 20, 11, 5, 15, 15, 18, 19, 14]
Assignments: 0
Consistency checks: 0
-----------------------------------------------
Simple Backtracking
Solved State:
Grid:
1 7 6 3 0 5 9 2 8 4
6 0 9 8 1 3 4 7 5 2
1 3 5 0 4 7 2 9 6 8
Column sums: [8, 10, 20, 11, 5, 15, 15, 18, 19, 14]
Assignments: 40
Consistency checks: 313
Elapsed time: 0.000455 seconds
-----------------------------------------------
Forward Checking
Solved State:
Grid:
1 7 6 3 0 5 9 2 8 4
6 0 9 8 1 3 4 7 5 2
1 3 5 0 4 7 2 9 6 8
Column sums: [8, 10, 20, 11, 5, 15, 15, 18, 19, 14]
Assignments: 41
Consistency checks: 236
Elapsed time: 0.000553 seconds
-----------------------------------------------
Forwar